In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
true_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

In [ ]:
fake_df.head()
true_df.head()

In [ ]:
true_df['check']='True'
fake_df['check']='Fake'

In [ ]:
true_df.head()

In [ ]:
true_df.describe()

In [ ]:
news_df = pd.concat([true_df, fake_df])

In [ ]:
news_df.head()

In [ ]:
news_df.info()

In [ ]:
news_df.sample(frac=1)

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

In [ ]:
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
news_df['article']=news_df['title']+""+news_df['text']+""+news_df['subject']

In [ ]:
df=news_df[['article','check']]

In [ ]:
df['article'] = df['article'].apply(lambda x: x.lower())

In [ ]:
import string

def punctuation_remover(dirty_string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punc=""
    for char in dirty_string:
        if char not in punctuations:
            no_punc= no_punc+char
    return no_punc
    

In [ ]:
df['article']=df['article'].apply(punctuation_remover)
df.head()

In [ ]:
print(df[df['check']=='True'].size)
print(df[df['check']=='Fake'].size)

In [ ]:
data=df.sample(frac=1)

In [ ]:
data.head()

In [ ]:
max_words=8000
tokenizer=Tokenizer(num_words=max_words,split=' ')
tokenizer.fit_on_texts(data['article'].values)
X=tokenizer.texts_to_sequences(data['article'].values)
X=pad_sequences(X, dtype='int32', padding='post')

In [ ]:
embed_dim=128
lstm_out=64

tf.keras.backend.clear_session()

with tpu_strategy.scope():
    model=tf.keras.Sequential()
    model.add(Embedding(max_words, embed_dim, input_length= X.shape[1]))
    model.add(Bidirectional(LSTM(lstm_out,dropout=0.2, return_sequences=True))
    model.add(Bidirectional(LSTM(lstm_out/2,))))
    model.add(Dense(64, activation='relu'))
    model.add(Dr)
    model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.9) ,metrics = ['accuracy'])
print(model.summary())

In [ ]:
#test-train split

Y=pd.get_dummies(data['check']).values
X_train,X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, random_state= 40)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)



In [ ]:
X_train = tf.cast(X_train, tf.int32)
Y_train=tf.cast(Y_train,tf.int32)

In [ ]:
batch_size= 16*tpu_strategy.num_replicas_in_sync
model.fit(X_train,Y_train,epochs=15, batch_size=batch_size,verbose=1)

In [ ]:
model.save_weights('./model.h5', overwrite=True)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

In [ ]:
from tensorflow.keras import layers

In [ ]:
tf.keras.backend.clear_session()

embed_dim=128
lstm_out=196

with tpu_strategy.scope():
    model2=tf.keras.Sequential()
    model2.add(Embedding(max_words, embed_dim, input_length= X.shape[1]))
    model2.add(SpatialDropout1D(0.2))
    model2.add(layers.Bidirectional(LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2)))
    model2.add(Dense(2, activation='softmax'))
    model2.compile(loss = 'categorical_crossentropy', optimizer='adam' ,metrics = ['accuracy'])
print(model2.summary())

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
batch_size= 16*tpu_strategy.num_replicas_in_sync
history=model2.fit(X_train,Y_train,epochs=7, batch_size=batch_size,verbose=1, callbacks=[callback])

In [ ]:
model2.save_weights('./model2.h5', overwrite=True)

In [ ]:
X_test = tf.cast(X_test, tf.int32)
Y_test = tf.cast(Y_test,tf.int32)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
dev_size=2000
X_dev = X_test[-dev_size:]
Y_dev = Y_test[-dev_size:]

X_test = X_test[:-dev_size]
Y_test = Y_test[:-dev_size]

score,acc=model2.evaluate(X_test, Y_test, verbose=2,batch_size=batch_size)

print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


In [ ]:
twt = ['The sweeping Republican tax bill signed into law by U.S. President Donald Trump on Friday introduces a cap, of $10,000, on deductions of state and local income and property taxes, known as SALT. The tax overhaul was the party’s first major legislative victory since Trump took office in January.The SALT provision will hit many taxpayers in states with high incomes, high property values and high taxes, like New York, New Jersey and California. Those states are generally Democratic leaning.“I’m not even sure what they did is legally constitutional and that’s something we’re looking at now,” Cuomo said in an interview with CNN.In an interview with CNBC, Cuomo suggested why the bill may be unconstitutional.']

twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=120, dtype='int32', value=0)
print(twt)
checknews = model2.predict(twt,batch_size=2,verbose = 2)[0]
if(np.argmax(checknews) == 1):
    print("false")
elif (np.argmax(checknews) == 0):
    print("True")